In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import yaml
 
try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("Yaml configuration file not found!")

df_full = pd.read_pickle(config['data']['clean']['df_full'])

In [8]:
df_full.head()

,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,5.0,60.0,79.0,U,2.0,189023.86,1.0,4.0
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,5.0,60.0,79.0,U,2.0,189023.86,1.0,4.0
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test,5.0,60.0,79.0,U,2.0,189023.86,1.0,4.0
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test,5.0,60.0,79.0,U,2.0,189023.86,1.0,4.0
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test,5.0,60.0,79.0,U,2.0,189023.86,1.0,4.0


In [17]:
new_df = df_full[["visitor_id", "Variation"]]
new_df.tail(20)

,visitor_id,Variation
755385,388766751_9038881013,NaN
755386,388766751_9038881013,NaN
755387,388766751_9038881013,NaN
755388,388766751_9038881013,NaN
755389,388766751_9038881013,NaN
755390,388766751_9038881013,NaN
755391,388766751_9038881013,NaN
755392,388766751_9038881013,NaN
755393,388766751_9038881013,NaN
755394,388766751_9038881013,NaN


In [ ]:
# 947159805_81558194550 NaN
# 580560515_7732621733 Test
# 762728880_76361333336 Control